## Reading Data From CSV

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)

In [ ]:
data = pd.read_csv("/mnt/z/Projects/Supply chain analysis/Data/DataCoSupplyChainDataset.csv", encoding='ISO-8859-1', low_memory=False)

### Data Transformation

In [ ]:
data.columns=data.columns.str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
data.columns

In [ ]:
columns_to_drop = ['customer_email','order_zipcode','customer_password', 'product_description', 
    'product_image', 'customer_street']

In [ ]:
data.drop(columns=[c for c in columns_to_drop if c in data.columns], inplace=True)
data.columns

## Checking NULLS

In [ ]:
data.isna().sum()

## Checking duplicated data

In [ ]:
data.duplicated().sum()

In [ ]:
data['full_name']=data['customer_fname'] + ' ' + data['customer_lname']
data.drop(columns=['customer_fname' ,'customer_lname'],inplace=True)


In [ ]:
data.rename(columns={'full_name' : 'customer_name'},inplace=True)


## Loading Data to SQL Server

In [ ]:
import os
import urllib
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get variables from .env
server = os.getenv("DB_SERVER")
database = os.getenv("DB_NAME")
username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
driver = os.getenv("DB_DRIVER")

# Build connection string
connection_str = (
    f"DRIVER={driver};"
    f"SERVER={server},1433;"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"LoginTimeout=30;"
    f"ConnectRetryCount=3;"
    f"ConnectRetryInterval=10;"
)

# Encode connection string
params = urllib.parse.quote_plus(connection_str)

# Create engine
engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={params}",
    fast_executemany=True
)

try:
    with engine.begin() as connection:
        data.to_sql(
            'fact_supply_chain',
            connection,
            if_exists='replace',   
            index=False
        )

    print(" Data successfully loaded to SQL Server.")

except Exception as e:
    print(" Connection Failed!")
    print(f" Error Details: {e}")